`conda activate CASM_TORCH2`

# Install torch if necessary. See https://pytorch.org/get-started/locally/
# pip install torch==1.11.0

# Install torch geometric if necessary. See: https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html
# pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.11.0+cpu.html


In [4]:
import sys
sys.path.append('../')

import logging
logging.getLogger('matplotlib').setLevel(logging.CRITICAL)
logging.getLogger('graphein').setLevel(logging.INFO)

In [30]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import pytorch_lightning as pl
from tqdm.notebook import tqdm
import networkx as nx
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import f1_score

import math
import glob

import warnings
warnings.filterwarnings("ignore")

In [18]:
from CASM.kin_sub_pairs import load_kin_sub_list
df = load_kin_sub_list(
    "../datasets/Kinase_Substrate_Dataset"
)
#print(df)

# Filter for only ``str`` rows
def filter_fn(row):
    if type(row['KIN_ACC_ID']) == str: 
        return True
    else: return False
    
df.apply(filter_fn, axis=1)

#y = torch.randn([5, 10])



tensor([[-0.1810, -0.1420,  0.4877, -0.8634,  0.2543,  0.3596, -0.4206, -1.6959,
          0.5501,  0.2975],
        [-0.1548, -1.1922, -0.4439,  0.0369, -0.3331,  0.2836,  0.7073,  0.7566,
          0.2760,  0.8032],
        [-0.0613, -0.9581,  0.1742,  0.2459, -0.9061,  0.9821,  0.9204, -0.7289,
         -0.2647, -0.2588],
        [ 1.0710, -0.3850,  1.0271,  0.7643,  1.6322,  0.5264,  0.1415, -0.3935,
         -0.2695,  0.9044],
        [-0.6352,  2.1997,  1.0185,  0.4732,  0.0593, -2.2480,  1.1534,  1.0344,
         -0.7669, -2.0264]])


In [53]:
npy_file = "../../DATA/PPI/npy_file_human_dataset.npy"
processed_dir = "../../DATA/processed"
npy_ar = np.load(npy_file)

print(npy_ar.shape)

from torch.utils.data import Dataset as Dataset_n
from torch_geometric.data import DataLoader as DataLoader_n

class LabelledDataset(Dataset_n):
    def __init__(self, npy_file, processed_dir):
      self.npy_ar = np.load(npy_file)
      self.processed_dir = processed_dir
      self.protein_1 = self.npy_ar[:,2]
      self.protein_2 = self.npy_ar[:,5]
      self.label = self.npy_ar[:,6].astype(float)
      self.n_samples = self.npy_ar.shape[0]

      print(f"P1: {self.protein_1[0:10]}")

    def __len__(self):
      return(self.n_samples)

    def __getitem__(self, index):
      prot_1 = os.path.join(self.processed_dir, self.protein_1[index]+".pt")
      prot_2 = os.path.join(self.processed_dir, self.protein_2[index]+".pt")
      #print(f'Second prot is {prot_2}')
      prot_1 = torch.load(glob.glob(prot_1)[0])
      #print(f'Here lies {glob.glob(prot_2)}')
      prot_2 = torch.load(glob.glob(prot_2)[0])
      return prot_1, prot_2, torch.tensor(self.label[index])


dataset = LabelledDataset(npy_file = npy_file ,processed_dir= processed_dir)

print(dataset[0])

final_pairs =  np.load(npy_file)
size = final_pairs.shape[0]

seed = 42
torch.manual_seed(seed)

trainset, testset = torch.utils.data.random_split(dataset, [math.floor(0.8 * size), size - math.floor(0.8 * size) ])


# Dataloaders
trainloader = DataLoader_n(dataset= trainset, batch_size= 4, num_workers = 0)
testloader = DataLoader_n(dataset= testset, batch_size= 4, num_workers = 0)
print("Length")
print(len(trainloader))
print(len(testloader))





(22217, 7)
P1: ['1FLK' '2JVX' '3ALN' '1J0W' '5FQD' '4EGC' '2CYX' '1CFH' '3UXG' '1P9C']


IndexError: list index out of range

In [54]:

import torch
from graphein.ml import ProteinGraphDataset
import graphein.protein as gp

To use the Graphein submodule graphein.protein.features.sequence.embeddings, you need to install: biovec 
biovec cannot be installed via conda
To use the Graphein submodule graphein.protein.visualisation, you need to install: pytorch3d 
To do so, use the following command: conda install -c pytorch3d pytorch3d


[10/30/22 19:41:07] WARNING  To use the Graphein submodule graphein.protein.meshes, you need to        ]8;id=233760;file:///home/cim/anaconda3/envs/CASM_TORCH2/lib/python3.10/site-packages/graphein/protein/meshes.py\meshes.py]8;;\:]8;id=594025;file:///home/cim/anaconda3/envs/CASM_TORCH2/lib/python3.10/site-packages/graphein/protein/meshes.py#29\29]8;;\
                             install: pytorch3d                                                                    
                             To do so, use the following command: conda install -c pytorch3d pytorch3d             

In [ ]:
"""
Kinase dataset
"""

npy_file = "../CASM/alphafold_structures" # TODO:SAVED ARRAY OF INTERACTING PROTEIN PAIRS
processed_dir = "../../"
npy_ar = np.load(npy_file)


print(npy_ar.shape)

from torch.utils.data import Dataset 
from torch_geometric.data import DataLoader 

class KinaseInteractionDataset(Dataset):
    def __init__(self, npy_file, processed_dir):

      # TODO
      self.npy_ar = np.load(npy_file)
      self.processed_dir = processed_dir
      self.kinase = self.npy_ar[:,2]
      self.substrate = self.npy_ar[:,5]

      self.mod_rsd = self.npy_ar[:,6]

      self.label = self.npy_ar[:,6].astype(float)
      self.n_samples = self.npy_ar.shape[0]

      print(f"kinase: {self.kinase[0:10]}")

    def __len__(self):
      return(self.n_samples)

    def __getitem__(self, index):
      

      kinase = os.path.join(self.processed_dir, self.kinase[index]+".pt")
      site = os.path.join(self.processed_dir, self.substrate[index] + "-" + self.mod_rsd[index] + ".pt") # Filenames are {acc_id}-{mod_rsd}.pt


      kinase = torch.load(glob.glob(kinase)[0])

      psite = torch.load(glob.glob(psite)[0])

      return kinase, psite, mod_rsd, torch.tensor(self.label[index])

      # TODO: alternative to keeping labels stored:  in __getitem__ , we lookup from another source whether or not 
      # the two things interact (i.e. which label); since this will change based on how we create negative examples etc. 
      # so we should abstract away into a function that returns what the label is
      # this means we don't regenerate the .npy array everytime we modify our dataset

dataset = KinaseInteractionDataset(npy_file=npy_file ,processed_dir=processed_dir)

print(dataset[0])

final_pairs =  np.load(npy_file)
size = final_pairs.shape[0]

seed = 42
torch.manual_seed(seed)

trainset, testset = torch.utils.data.random_split(dataset, [math.floor(0.8 * size), size - math.floor(0.8 * size) ])


# Dataloaders
trainloader = DataLoader(dataset=trainset, batch_size=4, num_workers = 0)
testloader = DataLoader(dataset=testset, batch_size=4, num_workers = 0)
print("Length")
print(len(trainloader))
print(len(testloader))


In [52]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from tqdm import tqdm
import pathlib
import math
import sklearn
#import torch_optimizer as optim
from torch.optim.lr_scheduler import MultiStepLR, ReduceLROnPlateau
#from metrics import *



for count, (p1, p2, label) in enumerate(trainloader):

    print(label)

IndexError: list index out of range

In [2]:
import torch

/home/cim/anaconda3/envs/CASM_TORCH2/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
import numpy as np

graph_path = "../GRAPHS2/processed/O13016.pt"

os.path.exists(graph_path)

g = torch.load(graph_path)
g
# #hot = g.amino_acid_one_hot
# #g

# exit(1)

# t = torch.as_tensor(np.array(g.amino_acid_one_hot))
# type(t)
# type(np.array(g.psi))
# t

# a = torch.as_tensor([1, 0, 0, 0])

# #a.resize_(4, 1)
# print(a.shape == torch.Size([4]))
# a = a.view(-1, 1)
# #torch.transpose(a, 1, 0)
# print(a.shape)
# a.dtype

Data(edge_index=[2, 762], node_id=[434], coords=[1], amino_acid_one_hot=[434], asa=[434], num_nodes=434)

In [51]:
from sklearn.metrics import accuracy_score
accuracy_score

<function sklearn.metrics._classification.accuracy_score(y_true, y_pred, *, normalize=True, sample_weight=None)>

In [55]:
import glob 
import re

processed_dir = "../GRAPHS1/processed/"

kin_filenames = glob.glob("KIN_*", root_dir=processed_dir)

kin_ids = []
for k in kin_filenames:
    m = re.search(r'KIN_(.*?).pt', k)
    kin_ids.append(m.group(1))

kin_ids

['Q9NWZ3',
 'P16234',
 'Q92630',
 'P32298',
 'P06213',
 'Q32MK0',
 'Q15831',
 'O95747',
 'P48729',
 'P54756',
 'P29317',
 'Q08881',
 'Q9BZL6',
 'P17612',
 'Q9UF33',
 'P45985',
 'O75116',
 'Q06418',
 'O43353',
 'Q96KB5',
 'P09769',
 'Q8IV63',
 'O95835',
 'Q9NZJ5',
 'P15056',
 'O94806',
 'P51956',
 'Q9UGJ0',
 'Q5TCX8',
 'Q7KZI7',
 'Q07912',
 'Q13131',
 'Q6P3R8',
 'P09619',
 'Q8TD08',
 'P51955',
 'P00533',
 'Q02750',
 'Q96GD4',
 'O00238',
 'Q13043',
 'P45983',
 'O96013',
 'O14936',
 'Q9P1W9',
 'Q8N5S9',
 'Q9Y2U5',
 'Q9UK32',
 'Q9NSY1',
 'P27448',
 'Q3MIX3',
 'P19784',
 'O94921',
 'O00443',
 'Q8TDX7',
 'P05129',
 'P50750',
 'P07949',
 'Q8IVH8',
 'P54764',
 'P11021',
 'Q15375',
 'P35968',
 'P36507',
 'P24941',
 'P43405',
 'P36888',
 'Q13177',
 'P30530',
 'Q96QP1',
 'P08631',
 'P11802',
 'Q8IYT8',
 'P42680',
 'Q16513',
 'Q05513',
 'O00141',
 'Q16659',
 'Q96GX5',
 'P42345',
 'P53779',
 'P45984',
 'Q9H2G2',
 'Q15759',
 'Q13188',
 'P50613',
 'Q9UBE8',
 'Q9Y572',
 'P52564',
 'P48730',
 'Q13555',

In [56]:
import plotly.express as px

fig = px.imshow([[1, 20, 30],
                 [20, 1, 60],
                 [30, 60, 1]])
fig.show()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/cim/anaconda3/envs/CASM_TORCH2/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_17442/527939474.py", line 6, in <cell line: 6>
    fig.show()
  File "/home/cim/anaconda3/envs/CASM_TORCH2/lib/python3.10/site-packages/plotly/basedatatypes.py", line 3398, in show
    return pio.show(self, *args, **kwargs)
  File "/home/cim/anaconda3/envs/CASM_TORCH2/lib/python3.10/site-packages/plotly/io/_renderers.py", line 396, in show
    raise ValueError(
ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/cim/anaconda3/envs/CASM_TORCH2/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 1993, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/cim/

In [9]:
import numpy as np
import torch 

idx = [5, 20, 43, 59]

node_id = [['A:PRO:697', 'A:ALA:747', 'A:LEU:704', 'A:LEU:702', 'A:GLU:696', 'A:SER:700', 'A:PRO:693', 'A:GLN:750', 'A:LEU:751', 'A:ARG:699', 'A:LEU:695', 'A:VAL:698', 'A:ARG:703', 'A:LEU:705', 'A:SER:701'], ['A:LEU:229', 'A:ALA:205', 'A:ARG:228', 'A:ARG:235', 'A:CYS:239', 'A:SER:234', 'A:LEU:244', 'A:CYS:243', 'A:PHE:232', 'A:LYS:238', 'A:ASP:240', 'A:PRO:241', 'A:MET:202', 'A:VAL:236', 'A:LEU:233', 'A:SER:201', 'A:ARG:231', 'A:ILE:237', 'A:ASP:242', 'A:ARG:245', 'A:GLY:206', 'A:THR:230'], ['A:LEU:758', 'A:ASP:761', 'A:GLN:776', 'A:ARG:772', 'A:GLU:769', 'A:LYS:760', 'A:TYR:774', 'A:LEU:763', 'A:GLY:770', 'A:ARG:764', 'A:VAL:779', 'A:ILE:777', 'A:ASN:759', 'A:ALA:771', 'A:THR:778', 'A:SER:775', 'A:GLN:780', 'A:GLU:762', 'A:PHE:756', 'A:VAL:773', 'A:LEU:755'], ['A:ARG:48', 'A:SER:53', 'A:LYS:50', 'A:THR:51', 'A:MET:52', 'A:TRP:49', 'A:LYS:55', 'A:GLU:56', 'A:LYS:57', 'A:GLY:58', 'A:ALA:54']]

n = []
for i in node_id:
    n += i 

n = np.asarray(n)
n = n.flatten()

for i in idx:
    print(n[i])



A:SER:700
A:SER:234
A:TYR:774
A:SER:53


ImportError: cannot import name 'Mapping' from 'collections' (/home/cim/anaconda3/envs/CASM1/lib/python3.10/collections/__init__.py)

In [47]:
"""Graphein dataset download method"""

"""ALL IMPORTS """

import sys 

from graphein.protein.config import ProteinGraphConfig
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import pytorch_lightning as pl
from tqdm.notebook import tqdm
import networkx as nx
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import f1_score

import math
import glob

import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn.functional as F

from torch_geometric.data import Batch, Data

from graphein.ml import ProteinGraphDataset

from CASM.kinase_dataset import KinaseSubstrateDataset

from graphein.ml.conversion import GraphFormatConvertor
import graphein.protein as gp

from graphein.protein.config import DSSPConfig
from graphein.protein.subgraphs import extract_surface_subgraph
from graphein.protein.features.nodes import rsa

from graphein.protein.features.sequence.embeddings import esm_sequence_embedding #, biovec_sequence_embedding

from graphein.protein.edges.distance import *

from graphein.protein.features.nodes.dssp import asa, phi, psi # Node metadata adding functions

from functools import partial
from graphein.protein.edges.distance import add_distance_threshold



In [10]:


# Try loading 



"""
Node features
"""
from graphein.protein.features.nodes.amino_acid import meiler_embedding, amino_acid_one_hot

from graphein.protein.graphs import construct_graph
config = ProteinGraphConfig(
    graph_metadata_functions=[esm_sequence_embedding],
)
g = construct_graph(config=config, pdb_code="3eiy")
em = g.graph["esm_embedding_A"]

len(g.nodes())
em[0]

-0.022799345

In [48]:
processed_dir = "../GRAPHS3_MEILER/processed/"

ids = ["Q9CX34.pt", "O75376.pt"]

datalist = [torch.load(os.path.join(processed_dir, i)) for i in ids]




phosphosite: Batch = Batch.from_data_list(datalist)
print(phosphosite.meiler[0])
num_nodes = phosphosite.num_nodes
residue_encoding = phosphosite.meiler 
features = [residue_encoding, phosphosite.asa, phosphosite.b_factor]   # , pro2_data.asa, pro2_data.phi, pro2_data.psi]
                
# Concatenate along 1st dim (dim=0) to get all nodes for all batches in one tensor
for i, f in enumerate(features):
        
    features[i] = torch.as_tensor(np.concatenate([np.array(i, dtype=np.float32) for i in f]))
    f = features[i]
    # Vertically stack if only one feature long
    if f.shape == torch.Size([num_nodes]):
        features[i] = f.view(-1, 1)

# Concatenate all features together
x = torch.cat(features, dim=1)
x = x.float()
x[0]

#type(torch.tensor(g.meiler))
#e = list(g.esm_embedding[0])
# e = list(g.meiler[0])
# len(e)

# m = nn.Softmax(dim=1)
# ms = nn.Sigmoid(dim=1)

# input = torch.randn(4, 20)
# ms(input)




# output = m(input)
# sum(output[0])

[dim_1    2.35
dim_2    0.22
dim_3    4.43
dim_4    1.23
dim_5    5.71
dim_6    0.38
dim_7    0.32
Name: MET, dtype: float64, dim_1    1.28
dim_2    0.05
dim_3    1.00
dim_4    0.31
dim_5    6.11
dim_6    0.42
dim_7    0.23
Name: ALA, dtype: float64, dim_1    1.28
dim_2    0.05
dim_3    1.00
dim_4    0.31
dim_5    6.11
dim_6    0.42
dim_7    0.23
Name: ALA, dtype: float64, dim_1    1.28
dim_2    0.05
dim_3    1.00
dim_4    0.31
dim_5    6.11
dim_6    0.42
dim_7    0.23
Name: ALA, dtype: float64, dim_1    1.28
dim_2    0.05
dim_3    1.00
dim_4    0.31
dim_5    6.11
dim_6    0.42
dim_7    0.23
Name: ALA, dtype: float64, dim_1    1.28
dim_2    0.05
dim_3    1.00
dim_4    0.31
dim_5    6.11
dim_6    0.42
dim_7    0.23
Name: ALA, dtype: float64, dim_1    0.00
dim_2    0.00
dim_3    0.00
dim_4    0.00
dim_5    6.07
dim_6    0.13
dim_7    0.15
Name: GLY, dtype: float64, dim_1    2.67
dim_2    0.00
dim_3    2.72
dim_4    0.72
dim_5    6.80
dim_6    0.13
dim_7    0.34
Name: PRO, dtype: float64,

tensor([2.3500e+00, 2.2000e-01, 4.4300e+00, 1.2300e+00, 5.7100e+00, 3.8000e-01,
        3.2000e-01, 2.4900e+02, 3.9020e+01])

In [56]:
idx = 4 
idx = 10 
from CASM.load_dbPTM import NUM_KINASE_FAMILIES, KINASE_TO_INDEX, KINASE_FAMILIES


#F.one_hot(, num_classes=NUM_KINASE_FAMILIES)
def get_1hot_kinase(k: str):
    if k not in KINASE_TO_INDEX:
        raise KeyError(f"'{k}' not a kinase family")

    idx = KINASE_TO_INDEX[k]
    return F.one_hot(torch.tensor(idx), num_classes=NUM_KINASE_FAMILIES)

k = "GRK"
get_1hot_kinase(k)



tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])

In [42]:
F.one_hot(torch.tensor(24), num_classes=NUM_KINASE_FAMILIES)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1])

In [233]:
# Get hit @ 1, hit @ 3 etc. accuracy 

# imagine 4 batches of predictions, each with 6 classes

# Given batch of predictions, and batch of labels: returns accuracy 
def hit_at_n_accuracy():
    pass 
    # TODO

"""
Returns true if (any) true kinase is in top N classes
"""
def hit_at_n(
    prediction, 
    true_kinases,  # Should be list of lists; for batch_size 
    n: int = 1, 
):
    num_hits = 0
    indexes = torch.topk(prediction, n).indices

    for i, idx in enumerate(indexes):
        
        for j in idx:
            j = int(j)
            kin = KINASE_FAMILIES[j]
            #print(f"kin: {kin}")
            if kin in true_kinases[i]:
                
                #print("yes!")
                #return True
                num_hits += 1

    return num_hits


batch_size  = 10^5
N           = 1

a = torch.randn(batch_size, NUM_KINASE_FAMILIES)
#print(a)
true_kinases = [["CDK", "CAMK2"], ["PKG"], ["PKG"], ["PKG", "CAMK2"]] # "Correct", if ANY of these are predicted in top N classes
true_kinases = [["CDK", "CAMK2"]] * batch_size

true_kinases = [["PKN"]] * batch_size
#true_kinases = [["PKN", "CAMK2", "CDK"]] * batch_size
num_correct = hit_at_n(a, true_kinases, n=N)

accuracy = num_correct / batch_size
accuracy

0.06666666666666667